# Energy Score

In [ ]:
def Energy_Score(energy_beta, observations_y, simulations_Y):

    m = len(simulations_Y)

    # First part |Y-y|. Gives the L2 dist scaled by power beta. Is a vector of length n/one value per location.
    diff_Y_y = torch.pow(
        torch.norm(
            (observations_y.unsqueeze(1) -
            simulations_Y.unsqueeze(0)).float(),
            dim=2,keepdim=True).reshape(-1,1),
        energy_beta)

    # Second part |Y-Y'|. 2* because pdist counts only once.
    diff_Y_Y = 2 * torch.pow(
        nn.functional.pdist(simulations_Y),
        energy_beta)
    Energy = 2 * torch.mean(diff_Y_y) - torch.sum(diff_Y_Y) / (m * (m - 1))
    return Energy

def SR_eval_E(energy_beta, real_samples, fake_samples):
    out = 0
    for i in range(len(real_samples)):
        out += Energy_Score(energy_beta, real_samples[i].view(-1,1), fake_samples[i].view(-1,1))
    return out / (len(real_samples) + len(fake_samples)) 


# Variogram Score

In [ ]:
def variogram(obs, sims, dist_mat, p):
    '''
    Variogram score from eq 10 in Pacchiardi paper [2024 Probabilistic Forecasting with Generative Networks
    via Scoring Rule Minimization] in JMLR.

    SR = sum_ij w_ij * (|y_i - y_j|^p - |Y_k,i - Y_k,j|^p)^2
    '''
    n = obs.size(0)
    m = sims.size(0)
    w_ij = torch.nan_to_num(1 / dist_mat, nan=0, posinf=0, neginf=0)
    diff_y_y = torch.abs(obs.unsqueeze(1) - obs) ** p
    diff_Y_Y = torch.abs(sims.unsqueeze(2) - sims.unsqueeze(1)).pow(p).mean(dim=0)

    SR = w_ij * (diff_y_y - diff_Y_Y) ** 2
    return SR.sum().item()


def SR_eval_V(real_samples, fake_samples, distance_matrix, p):
    vario_ = 0
    for k in range(len(real_samples)):
        multi = MultivariateNormal(torch.zeros(25), covariance_matrix=cov_.float())
        sims = multi.sample((100,))
        vario_ = torch.tensor(vario_) + variogram(real_samples[k], fake_samples, dist_mat, 1)
    return vario_

# Energy-Variogram Score

In [ ]:
def SR_eval_EV(real_samples, fake_samples, energy_beta, distance_matrix, p):
    a1 = 1  #weights of Energy and Variogram 
    a2 = 1 / 20000
    SR_eval_V_result = SR_eval_V(real_samples, fake_samples, distance_matrix, p)  
    SR_eval_E_result = SR_eval_E(energy_beta, real_samples, fake_samples)  
    return (a1* SR_eval_E_result)  + (a2 * SR_eval_V_result)